- 수원 및 수원 인근 소재 대학교

In [3]:
import requests, json
from urllib.parse import quote

In [4]:
with open('data/roadapikey.txt') as f:
    road_key = f.read()

In [5]:
univ_list = '아주대학교, 동남보건대학, 수원 경기대학교, 수원 성균관대학교, 수원여자대학, 용인 경희대학교, 수원대학교, 수원과학대학, 장안대학, 협성대학, 수원 한국방송통신대학교'.split(', ')
univ_list

['아주대학교',
 '동남보건대학',
 '수원 경기대학교',
 '수원 성균관대학교',
 '수원여자대학',
 '용인 경희대학교',
 '수원대학교',
 '수원과학대학',
 '장안대학',
 '협성대학',
 '수원 한국방송통신대학교']

In [6]:
road_addr_list = []
for univ in univ_list:
    base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
    params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
    params2 = f'keyword={quote(univ)}&resultType=json'
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text[1:-1])
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [7]:
df = pd.DataFrame({
    '대학명': list(map(lambda x: x.split()[-1], univ_list)), '도로명주소': road_addr_list
})
df

,대학명,도로명주소
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동)
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동)
2,경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동)
3,성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동)
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동)
5,경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동)
6,수원대학교,경기도 화성시 봉담읍 와우안길 17
7,수원과학대학,경기도 화성시 정남면 세자로 288
8,장안대학,경기도 화성시 봉담읍 삼천병마로 1182
9,협성대학,경기도 화성시 봉담읍 최루백로 72


In [8]:
with open('data/kakaoapikey.txt') as f:
    kakao_key = f.read()

In [9]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [10]:
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [11]:
colors = ['pink', 'lightgreen', 'darkgreen', 'black', 'lightred', 'darkblue', 'lightblue', 'purple', 'green', 'cadetblue', 'gray', 'red', 'blue', 'lightgray', 'darkpurple', 'orange', 'darkred', 'beige']
color = []
for i in range(df.index[-1]+1):
    color.append(colors[i])
color

['pink',
 'lightgreen',
 'darkgreen',
 'black',
 'lightred',
 'darkblue',
 'lightblue',
 'purple',
 'green',
 'cadetblue',
 'gray']

In [12]:
df['위도'] = lat_list
df['경도'] = lng_list
df['color'] = color
df

,대학명,도로명주소,위도,경도,color
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142,pink
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000,lightgreen
2,경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117,darkgreen
3,성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209,black
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동),37.252705,126.963307,lightred
5,경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199,darkblue
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480,lightblue
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927,purple
8,장안대학,경기도 화성시 봉담읍 삼천병마로 1182,37.213287,126.944482,green
9,협성대학,경기도 화성시 봉담읍 최루백로 72,37.213251,126.953623,cadetblue


In [13]:
df.to_csv('data/수원용인소재대학교.csv', index=False)
pd.read_csv('data/수원용인소재대학교.csv')

,대학명,도로명주소,위도,경도,color
0,아주대학교,경기도 수원시 영통구 월드컵로 199 (원천동),37.279205,127.044142,pink
1,동남보건대학,경기도 수원시 장안구 천천로74번길 50 (정자동),37.296938,126.987000,lightgreen
2,경기대학교,경기도 수원시 영통구 광교산로 154-42 (이의동),37.298014,127.033117,darkgreen
3,성균관대학교,경기도 수원시 장안구 서부로 2066 (천천동),37.295976,126.974209,black
4,수원여자대학,경기도 수원시 권선구 온정로 72 (오목천동),37.252705,126.963307,lightred
5,경희대학교,경기도 용인시 기흥구 덕영대로 1732 (서천동),37.239800,127.081199,darkblue
6,수원대학교,경기도 화성시 봉담읍 와우안길 17,37.210117,126.979480,lightblue
7,수원과학대학,경기도 화성시 정남면 세자로 288,37.192033,126.983927,purple
8,장안대학,경기도 화성시 봉담읍 삼천병마로 1182,37.213287,126.944482,green
9,협성대학,경기도 화성시 봉담읍 최루백로 72,37.213251,126.953623,cadetblue


In [14]:
# pushpin = folium.features.CustomIcon('data/pushpin.png', icon_size=(30,30))
# icon = pushpin

In [15]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=11)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.도로명주소[i], max_width=200),
        tooltip=df.대학명[i],
        icon=folium.Icon(
            color=df.color[i], icon_color='white',
            icon='glyphicon glyphicon-pencil',
            prefix='glyphicon'
        )
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">수원 및 수원 인근 소재 대학교</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map